In [2]:
import numpy as np
import matplotlib.pyplot as plt

# <center> Notes on Integration </center>

In this notes, we shall look at the following:

## <center> What does it mean to be correct? </center>

Consider the trapezoidal method of integration

In [3]:
def integrate_trapezoidal(f, eval_x, **kwargs):
    y = f(eval_x, **kwargs)
    h = eval_x[1] - eval_x[0]
    area = h*(np.sum(y)-0.5*y[0]-0.5*y[-1])
    return area

How can we determine whether or not the output of our integration method is correct? Say we wish to integrate,
\begin{equation}
y = a x^2
\end{equation}

In [4]:
def f(x,a):
    return a*x**2
xmin=-5
xmax=5
a=1
x_res=500
x_mesh=np.linspace(xmin, xmax, x_res)
integrate_trapezoidal(f, x_mesh, a=1)

83.33400267468905

Of course we are blessed with the knowledge that we already know the exact solution before-hand, which should be
\begin{equation}
\int_{xmin}^{xmax} a x^2 dx =  a \dfrac{xmax^3 - xmin^3}{3}
\end{equation}
which yields
\begin{equation}
\dfrac{250}{3}
\end{equation}

So, a naive answer here is that, we should just check. We have the luxury of being able to just cross-check to the exact answer.

83.33333333333333

83.33400267468905
83.333500333834
83.33337504169572
83.33333500030132


There are two problems with this method. There's a lot of trial and error, and a lot of eyeballing involved! These should be the job of your computer.

Second, most of the time, we won't know the exact solution before hand.

---

The use of exact solutions are important in troubleshooting and prototyping your code. In a controlled environment like this, it's easiest to find out where exactly your code is going wrong. Consider the following function,

In [9]:
def intrap_w(f, eval_x, **kwargs):
    y = f(eval_x, **kwargs)
    h = eval_x[1] - eval_x[0]
    area = h*(np.sum(y) -0.5*y[0] - 0.5*y[-1])
    return area

In [10]:
def f(x,a):
    return a*x**2
xmin=-5
xmax=5
a=1
x_res=500
x_mesh=np.linspace(xmin, xmax, x_res)
intrap_w(f, x_mesh, a=1)

83.33400267468905

Can you identify where things have gone wrong? What's interesting to note is that the answer is off from the correct answer by distance of around 25, which suspisciously looks like $5^2$. 

The error must be related to how we are handling the boundary points! And indeed, if we look at the wrong integration scheme, we have failed to enclose the boundary points inside parenthesis, so they are not scaled down by the step size $h$.

---

Let us add a wrinkle to our integration problem. Suppose we do not know the function before hand.

In [12]:
from unknownfuncs import f1

In [13]:
xmin = -5
xmax = 5
x_res = 100
x_mesh = np.linspace(xmin, xmax, x_res)
print(integrate_trapezoidal(f1,x_mesh))

4.784657420592104


Again, the best we can do is to simply compare the outputs of our integration scheme for different mesh sizes. One idea is to refine our mesh to include twice as many points, and then check their difference.

In [14]:
x_lowresmesh = np.linspace(xmin, xmax, x_res)
x_highresmesh = np.linspace(xmin, xmax, 2*x_res)
area_lowres = integrate_trapezoidal(f1, x_lowresmesh)
area_highres = integrate_trapezoidal(f1, x_highresmesh)
print(area_lowres-area_highres)

0.002259479369241646


It would seem that the area we calculated using the low resolution mesh is at the very least accurate to the tenths digit. We can, of course, refine our mesh even more

In [17]:
x_higherresmesh = np.linspace(xmin, xmax, 2*x_res)
area_higherres = integrate_trapezoidal(f1, x_higherresmesh)
print(area_lowres-area_higherres)
print(area_highres-area_higherres)

0.002259479369241646
0.0


Since comparing a mesh with quadruple the number of points and double the number of points yields a error that has the same order of magnitude (~10^-3), we can be pretty confident that the low resolution calculation is accurate to the hundredths digit.

Maybe we can automate this process.

## <center> Adaptive integration </center>

Now we wish to do the following: we wish to refine our integration mesh until the different between two adjacent mesh sizes yield a difference below a certain error threshold.

Instead of providing a constant mesh, we shall instead give our integrator enough information to construct arbitrarily refined meshes. This basically means we only need to provide it the bounds of our integration, and include a method inside the function to define meshes.

4.7816598692411
4.781659032457059
4.781658771024859
4.781658754695111
4.781658753886653
maximum mesh size reached. terminating.
None
maximum mesh size reached. terminating.
None


However, there are two things that can go wrong with this method.

8.335033840084363e+21
8.335033840084357e-19


What would happen if we tried to use our adaptive integration scheme for the above functions? Implicit in the design of our adaptive integration method is the assumption that we already know before hand the order of magnitude of the answer.

We were only providing epsilon with the assumption that the area is of order $10^0$. How do we change our algorithm to adapt to different sizes of the answer?

## <center> Absolute and relative errors </center>

A rough modification to our problem is that we can give the user a choice to check the error in its absolute magnitude (it's current implementation) and the error in its relative magnitude.|

8.333343521772304e+21
8.333335878452287e+21
8.333333492308706e+21
8.33333334326053e+21
8.333333335812055e+21
8.333333333507957e+21
maximum mesh size reached. terminating.
None


8.333343521772303e-19
8.333335878452285e-19
8.333333492308705e-19
8.333333343260529e-19
8.333333335812055e-19
8.3333333335079535e-19
maximum mesh size reached. terminating.
None


Now it seems a waste to check what error method we use everytime we refine the mesh.

maximum mesh size reached. terminating.
None


## <center> A caveat on relative and absolute errors </center>

Let us import one last unknown function which may break out method

/tmp/ipykernel_9833/3660577264.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if abs(error/area) < epsilon:


9.436953307911289e-16
maximum mesh size reached. terminating.
None
maximum mesh size reached. terminating.
None
maximum mesh size reached. terminating.
None
maximum mesh size reached. terminating.
None


What is this mysterious function that cannot be made to converge?

Of course, you are not supposed to know.

-7.123235446216637e-16

What is going on? Well, it seems like our integral is evaluating to zero! That is what is going on.

If the integral is zero, then how do we show that numerically? Know that this is a very difficult and subtle problem. When something vanishes, it usually means there's some interesting physics involve. Something is making terms cancel.

If you encounter this result in the wild, while doing research, the answer always depends on the problem you are working on. It would usually mean using both analytic and numerical methods to convince other people that what you are getting is actually zero and not some tiny number you cannot resolve with floating points.

## <center> Efficiency </center>

We note that in refining, we can recycle a lot of our work. Consider integrating on a mesh with points $x = 0, 2, 4$ and $x = 0, 1, 2, 3, 4$. Let $ h = 2$.

\begin{eqnarray}
A_1 &=& h \left( \dfrac{f(0)}{2} + f(2) + \dfrac{f(4)}{2} \right) \\
A_2 &=& \dfrac{h}{2} \left( \dfrac{f(0)}{2} + f(1) + f(2) + f(3) + \dfrac{f(4)}{2} \right)
\end{eqnarray}

Notice that
\begin{equation}
A_2 = \dfrac{A_1}{2} + \dfrac{h}{2} \left( f(1) + f(3) \right)
\end{equation}

Thus, we may actually recycle our calculation with a lower resolution mesh, and update it by rescaling the original answer and adding terms related to points unique to the higher resolution mesh.

4.781658757956817
4.781658758981339


16.2 ms ± 661 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


CPU times: user 9.89 ms, sys: 3.37 ms, total: 13.3 ms
Wall time: 20.6 ms


4.781658758981339

Does this speed up make any sense? Yes!

A large part of the calculation slow down rests at the evaluation of the function being integrated at the points. Notice that instead of calculating

\begin{equation}
N + 2N + 4N + \dots 2^k N = N \left( \dfrac{1 - 2^k}{1 - 2} \right) \approx 2^k N
\end{equation}

times, we are evaluating
\begin{equation}
N + (N-1) + 2(N-1) + \dots + 2^{k-1}(N-1) \approx 2^{k-1} N
\end{equation}

times in the better method. So we expect a speed up factor of around 2.

## <center> Integration using the midpoint rule </center>

Given a equally spaced mesh, the midpoint rule uses function evaluations at the midpoints between two successive points.

That is, say we start with a mesh with points $x = 0, 2, 4$, then we will instead use the mesh $x = 1, 3$ in integration.

\begin{equation}
A = h \left( f(1) + f(3) \right), \qquad h = 2
\end{equation}

There are two main advantages of the midpoint rule.
1. It is marginally more accurate than the trapezoidal rule.
2. It does not evaluate the integrand at the boundaries.\

The second point is crucial. Oftentimes, non-trivial parts of the integrand is found at the boundary points. As we shall see in the next section, we will usually encounter such problems that may only be resolved by an analytic application of L'hopital's rule. Unless you want to teach your integration scheme the basics of algebra and calculus, then it would be ideal if we can simply ignore boundary values as much as possible.

This puts integrate_trapezoidal and integrate_midpoint on two different levels. Whereas integrate_trapezoidal only does not require the creation of a mesh, integrate_midpoint at its current form does. Let us define the two so that they are more equivalant.

We may implement a similar thing to get_error_check(), but this time one that selects the proper integration method.